In [25]:
import json
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, select, MetaData, Table
import requests
import sqlalchemy as sa
import urllib
import sql_scripts as ss
from datetime import date, datetime, timedelta
from threading import Thread

In [26]:
def getAuthforWMS(f_data):
    wmsAccess = f_data["wmsAccess"][0]
    return wmsAccess

In [27]:
#gets connections
def getConnforMYSQL(f_data, accessType):
    dialect = pyodbc.drivers()[-1]
    server = f_data[accessType][0]["server"]
    db = f_data[accessType][0]["database"]
    uid = f_data[accessType][0]["uid"]
    pwd = f_data[accessType][0]["pwd"]
    driver = f_data[accessType][0]["dialect_driver"]
    port = f_data[accessType][0]["port"]

    if accessType == "azureAccess":
        connection_string = (
            " Driver={%s}" %dialect +
            "; SERVER=%s" %server + 
            "; Database=%s " %db + 
            "; UID=%s" %uid +
            "; PWD=%s" %pwd
        )
        quoted = urllib.parse.quote_plus(connection_string)
        quoted = f_data[accessType][0]["dialect_driver"] + quoted
        engine = create_engine(quoted, fast_executemany=True).execution_options(isolation_level="AUTOCOMMIT")
    else:
        quoted = driver + uid + ":" + pwd + "@" + server + ":" + str(port) + "/" + db
        engine = create_engine(quoted).execution_options(isolation_level="AUTOCOMMIT")
        
    return engine

In [28]:
#Extracts data from DB
def extractionFunction(conn_integrator, sql_text):
    with conn_integrator.begin():
        str_sql = sa.text(sql_text)
        results = conn_integrator.execute(str_sql)
        columns = results.keys()

        list_columns = []
        list_rows = []
        #print column_names
        for column_name in columns:
            list_columns += [column_name]
        for row in results:
            list_rows += [row]

    return list_rows, list_columns

In [29]:
#get response from API
def setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers):
    #adds default headers
    headers['Accept'] =  "application/json"
    headers['Content-Type'] =  "application/json"   

    completeHTTP = schemeHTTP + baseHTTP + extraHTTP
    response = requests.get(completeHTTP, headers=headers)
    
    return response

In [30]:
class threadMain:
    def __init__(self, index, columns_ordersList, rows_ordersList, conn_azure, file):
        self.subThreads = []
        self.file = file
        self.index = index
        self.columns_ordersList = columns_ordersList
        self.rows_ordersList = rows_ordersList
        self.conn_azure = conn_azure
        self.idPedido = self.getIdPedido(conn_azure)

        self.t = Thread(target=self.threadMainTask, args=( ))
        self.t.start()
    
    #gets ID_PEDIDO from the SEQ_ID_PEDIDO_VTEX and associates with the orderMainThread
    def getIdPedido(self, conn_azure):
        #rows_IdPedidoList, columns_IdPedidoList = extractionFunction(self.conn_azure, ss.queryList[self.file]['sql_querySequence'] )
        #return (rows_IdPedidoList[0])
        return 1

    def getThread(self):
        return (self.t)
    
    def setOrderId(self, orderId):
        self.orderId = orderId

    #shoudn't be necessary
    #def threadSubTask(self, pageNumber, schemeHTTP, baseHTTP, headers, yesterday, now):
    #    extraHTTP = "/api/oms/pvt/orders?per_page=100&page=%s&f_creationDate=creationDate:[%s TO %s]" %(pageNumber, yesterday, now)
    #    response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers)
    #    
    #    if "list" in response.json():
    #        globals()["dataList"] = globals()["dataList"] + response.json()["list"]
    #    else:
    #        print (response.json())
        
    def threadMainTask(self):
        #redeclare self variables
        index = self.index
        columns_ordersList = self.columns_ordersList
        rows_ordersList = self.rows_ordersList
        subThreads = self.subThreads
        #begin a MainThread
        print(f'Starting Thread index: {index}')
        
        for column_index in range(len(columns_ordersList)):
            if columns_ordersList[column_index] == "CONTA":
                account = rows_ordersList[index][column_index]
            if columns_ordersList[column_index] == "CHAVE":
                key = rows_ordersList[index][column_index]
            if columns_ordersList[column_index] == "TOKEN":
                token = rows_ordersList[index][column_index]
            if columns_ordersList[column_index] == "orderId":
                self.setOrderId(rows_ordersList[index][column_index])

        #now = datetime.now().strftime(ss.queryList[self.file]['nowStrftime'])
        #yesterday = (datetime.now() - timedelta(days=ss.queryList[self.file]['timedelta'])).strftime(ss.queryList[self.file]['yesterdayStrftime'])

        schemeHTTP = "https://"
        baseHTTP = "%s.%s.com.br" % (account, "vtexcommercestable")
        headers = {                
                'X-VTEX-API-AppKey': key,
                'X-VTEX-API-AppToken': token
                }
        pageNumber = 1
        extraHTTP = "/api/oms/pvt/orders/%s" %(self.orderId)

        #get first response
        response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers)

        print("response index: %s status_code: %s" %(self.index, response.status_code))
        #good Responses
        if response.status_code == 200:
            #include initial dataDict
            globals()["dataList"] = globals()["dataList"] + [response.json()]

        #Bad Responses
        else:
            globals()["errorList"].append({'account': account, 'status_code': response.status_code, 'time': datetime.now(), 'order' : self.orderId })
    

In [31]:
def main (file):
    #open auth file
    auth = open('auth.json')
    auth_load = json.load(auth)

    #getting conn for azure
    engine_azure = getConnforMYSQL(auth_load, "azureAccess")
    conn_azure = engine_azure.connect()

    #getData from azureDB about vTex Account list
    rows_ordersList, columns_ordersList = extractionFunction(conn_azure, ss.queryList[file]['sql_query'] )

    #list of main threads
    threads = []
    
    #start the dataList
    globals()["dataList"] = []

    #start the errorList
    globals()["errorList"] = []

    for index in range(len(rows_ordersList)):
        t = threadMain(index, columns_ordersList, rows_ordersList, file, conn_azure)
        threads.append(t.getThread())
    
    for t in threads:
        t.join()

    df_new = pd.DataFrame.from_dict(globals()["dataList"])
    df2_new = pd.DataFrame.from_dict(globals()["errorList"])
    
    #fillNan with empty string
    df_new = df_new.fillna("")
    df2_new = df2_new.fillna("")

    print (df_new)
    print (df2_new)

    #Gets pedidos_vtex to check weather the ID_PEDIDO is already in pedidos_vtex -> UPDATE information or INSERT in pedidos_vtex
        
    #df.to_sql(ss.queryList[file]['resultSuccessTable'], engine_azure, if_exists='replace', index=False)
    #if len(globals()["errorList"]) > 0:
    #    df2.to_sql(ss.queryList[file]['resultFailedTable'], engine_azure, if_exists='append', index=False)

    conn_azure.close()
    

In [32]:
if __name__ == "__main__":
    file = "getOrders_temptables"
    main(file)
    print('getOrders_temptables: done')

Starting Thread index: 0
Starting Thread index: 1
Starting Thread index: 2
Starting Thread index: 3
Starting Thread index: 4
Starting Thread index: 5
Starting Thread index: 6
Starting Thread index: 7
Starting Thread index: 8
Starting Thread index: 9
Starting Thread index: 10
Starting Thread index: 11
Starting Thread index: 12
Starting Thread index: 13
Starting Thread index: 14
Starting Thread index: 15
Starting Thread index: 16
Starting Thread index: 17
Starting Thread index: 18
Starting Thread index: 19
Starting Thread index: 20
Starting Thread index: 21
Starting Thread index: 22
Starting Thread index: 23
Starting Thread index: 24
Starting Thread index: 25
Starting Thread index: 26
Starting Thread index: 27
Starting Thread index: 28
Starting Thread index: 29
Starting Thread index: 30
Starting Thread index: 31
Starting Thread index: 32
Starting Thread index: 33
Starting Thread index: 34
Starting Thread index: 35
Starting Thread index: 36
Starting Thread index: 37
Starting Thread index: